## Import libraries and mount the drive

In [0]:
try:
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
import os
from IPython import display
import time
import math

print(tf.__version__)

from google.colab import drive
drive.mount('/content/drive')

TensorFlow 2.x selected.
2.1.0
Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import sys
sys.path.append('./drive/My Drive/Master_thesis/generation')
from dataset_helpers import load_dataset
from evaluation_metrics import compute_psnr, compute_ssim, compute_mse, compute_mse_tumor, compute_psnr_tumor

## Loading the validation and test sets

In [0]:
def load_datasets():
    validation = load_dataset(prefix_path + 'brats2015_validation_crop_mri.tfrecords', ['MR_T1', 'MR_T1c', 'MR_T2', 'MR_Flair'], batch_size=BATCH_SIZE, shuffle=False)
    training = load_dataset(prefix_path + 'brats2015_training_crop_mri.tfrecords', ['MR_T1', 'MR_T1c', 'MR_T2', 'MR_Flair'], batch_size=BATCH_SIZE, shuffle=True)
    testing = load_dataset(prefix_path + 'brats2015_testing_crop_mri.tfrecords', ['MR_T1', 'MR_T1c', 'MR_T2', 'MR_Flair'], batch_size=BATCH_SIZE, shuffle=True)
    return training, validation , testing

In [0]:
prefix_path = './drive/My Drive/Master_thesis/datasets/'
BATCH_SIZE = 32

training, validation, testing = load_datasets()

## Load some useful functions




In [0]:
def resize_with_crop(*args): # possibile arguments: input, gt, prediction and maybe the segmentation

    image0 = tf.image.resize_with_crop_or_pad(args[0], 155, 194)
    image1 = tf.image.resize_with_crop_or_pad(args[1], 155, 194)
    image2 = tf.image.resize_with_crop_or_pad(args[2], 155, 194)
    image3 = tf.image.resize_with_crop_or_pad(args[3], 155, 194)
    if len(args) == 5:      # crop also the segmentation, if is given as additional argument
        image4 = tf.image.resize_with_crop_or_pad(args[4], 155, 194)
        return image0, image1, image2, image3, image4
    if len(args) == 6:      # crop also the segmentation, if is given as additional argument
        image4 = tf.image.resize_with_crop_or_pad(args[4], 155, 194)
        image5 = tf.image.resize_with_crop_or_pad(args[5], 155, 194)
        return image0, image1, image2, image3, image4, image5
    if len(args) == 7:      # crop also the segmentation, if is given as additional argument
        image4 = tf.image.resize_with_crop_or_pad(args[4], 155, 194)
        image5 = tf.image.resize_with_crop_or_pad(args[5], 155, 194)
        image6 = tf.image.resize_with_crop_or_pad(args[6], 155, 194)
        return image0, image1, image2, image3, image4, image5, image6
    if len(args) == 8:      # crop also the segmentation, if is given as additional argument
        image4 = tf.image.resize_with_crop_or_pad(args[4], 155, 194)
        image5 = tf.image.resize_with_crop_or_pad(args[5], 155, 194)
        image6 = tf.image.resize_with_crop_or_pad(args[6], 155, 194)
        image7 = tf.image.resize_with_crop_or_pad(args[7], 155, 194)
        return image0, image1, image2, image3, image4, image5, image6, image7
    return image0, image1, image2, image3

In [0]:
# MASK THE SEGMENTATION with the prediciton and the ground_truth and then compute the metrics (even if there is the black pixels
# it should give me an idea about how much is going the generation of the tumor during the generation. of course the score
# will be influeced (biased) by all the black pixels.)

def retrieve_tumor_area(*args):  # ground_truth, prediction1, prediction2, segmentation, prediction3*
    prediction1_np = args[0].numpy()
    prediction2_np = args[1].numpy()
    segmentation_np = args[2].numpy()
    
    idx = (segmentation_np==0)      
    prediction1_np[idx] = segmentation_np[idx]
    prediction2_np[idx] = segmentation_np[idx]
    
    return prediction1_np, prediction2_np

## Mean normalization

![alt text](https://wikimedia.org/api/rest_v1/media/math/render/svg/5c591a0eeba163a12f69f937adbae5886d6273db)

In the paper they say: "Each patient scan is normalized by dividing each sequence by its mean intensity value. "
But the formula is taken from a lecture from Andrew Ng, where he defines the Mean normalization as in the formula above. (resource: https://www.youtube.com/watch?v=e1nTgoDI_m8)

See also: https://stats.stackexchange.com/questions/138046/normalizations-dividing-by-mean

In [0]:
def mean_rescale(x, xmin, xmax):
    mean = tf.reduce_mean(x)
    return ((x-mean)/(xmax-xmin)) 

In [0]:
@tf.function
def mean_normalize(image):
    image_normalized = tf.TensorArray(tf.float32, size=BATCH_SIZE)
    for i in range(BATCH_SIZE):
        # rescaling each image in the batch
        max_value = tf.math.reduce_max(image[i])
        min_value = tf.math.reduce_min(image[i])
        x = mean_rescale(image[i], min_value, max_value)
        image_normalized = image_normalized.write(i, x)
    image_normalized = image_normalized.stack()
    return image_normalized

## Discard black images from batch (put values to 'nan')

This normalization is just to test the metrics and see if there is a big difference in normalizing the prediction and the gt.
This method is used to normalize (and so put to 'nan') only the black images, while the other images of the batch are kept with the original values. This allows me to discard the black images in the computation of the metrics.

In [0]:
# this normalization is just to test the metrics and see if there is a big difference in normalizing the prediction and the gt.
# this method is used to normalize (and so put to 'nan') only the black images, while the other images of the batch are kept 
# with the original values. This allows me to discard the black images in the computation of the metrics.

def black_images_to_nan(image): 
    image_normalized = tf.TensorArray(tf.float32, size=BATCH_SIZE)
    for i in range(BATCH_SIZE):
        # rescaling each image in the batch
        max_value = tf.math.reduce_max(image[i])
        min_value = tf.math.reduce_min(image[i])

        # if the max = min most likely it's a black image (or an image without any important information)
        if tf.math.equal(max_value, min_value):        
            print("I found a black image! at index {}".format(i))
            x = mean_normalize(image[i], min_value, max_value)
            image_normalized = image_normalized.write(i, x)
        else:
            image_normalized = image_normalized.write(i, image[i])
    image_normalized = image_normalized.stack()
    return image_normalized

## Evaluate GAN

In [0]:
def evaluate_GAN(dataset, set_type, first_sequence, second_sequence, evaluate_tumor_area=False):
    num_batches = 0
    if set_type == 'test':
        num_batches = 112
    elif set_type == 'validation':
        num_batches = 108
    elif set_type == 'train':
        num_batches = 876
    
    #num_batches = 10
    container_psnr = tf.TensorArray(tf.float32, size=num_batches)     
    container_mse = tf.TensorArray(tf.float32, size=num_batches)
    container_ssim = tf.TensorArray(tf.float32, size=num_batches)
    
    if evaluate_tumor_area:
        container_psnr_tumor = tf.TensorArray(tf.float32, size=num_batches)     
        container_mse_tumor = tf.TensorArray(tf.float32, size=num_batches)
        container_ssim_tumor = tf.TensorArray(tf.float32, size=num_batches)
    
    for idx, (t1, t2, t1c, tflair, segmentation, patient) in dataset.enumerate():

        # crop the images
        t1_cr, t1c_cr, t2_cr, tflair_cr, segmentation_cr = resize_with_crop(t1, t1c, t2, tflair, segmentation)
        
        if first_sequence == 0:
            prediction_normalized_cr = mean_normalize(t1_cr)
        elif first_sequence == 1:
            prediction_normalized_cr = mean_normalize(t2_cr)
        elif first_sequence == 2:
            prediction_normalized_cr = mean_normalize(t1c_cr)
        elif first_sequence == 3:
            prediction_normalized_cr = mean_normalize(tflair_cr)

        if second_sequence == 0:
            ground_truth_normalized_cr = mean_normalize(t1_cr)
        elif second_sequence == 1:
            ground_truth_normalized_cr = mean_normalize(t2_cr)
        elif second_sequence == 2:
            ground_truth_normalized_cr = mean_normalize(t1c_cr)
        elif second_sequence == 3:
            ground_truth_normalized_cr = mean_normalize(tflair_cr)

        # compute the metrics of similarity
        mean, std, psnr = compute_psnr(ground_truth_normalized_cr, prediction_normalized_cr)
        container_psnr = container_psnr.write(idx, psnr)
        mean, std, mse = compute_mse(ground_truth_normalized_cr, prediction_normalized_cr)
        container_mse = container_mse.write(idx, mse)
        mean, std, ssim = compute_ssim(ground_truth_normalized_cr, prediction_normalized_cr)
        container_ssim = container_ssim.write(idx, ssim)

        if evaluate_tumor_area:
            ground_truth_masked_normalized, prediction_masked_normalized = retrieve_tumor_area(ground_truth_normalized_cr, prediction_normalized_cr, segmentation_cr)
            mean, std, psnr = compute_psnr_tumor(ground_truth_masked_normalized, prediction_masked_normalized)
            container_psnr_tumor = container_psnr_tumor.write(idx, psnr)
            mean, std, mse = compute_mse_tumor(ground_truth_masked_normalized, prediction_masked_normalized)
            container_mse_tumor = container_mse_tumor.write(idx, mse)
            mean, std, ssim = compute_ssim(ground_truth_masked_normalized, prediction_masked_normalized)
            container_ssim_tumor = container_ssim_tumor.write(idx, ssim)

       # idx += 1
    container_psnr = container_psnr.stack()
    container_mse = container_mse.stack()
    container_ssim = container_ssim.stack()
    mean_psnr = tf.reduce_mean(tf.boolean_mask((container_psnr), tf.math.is_finite(container_psnr)))
    std_psnr = tf.math.reduce_std(tf.boolean_mask((container_psnr), tf.math.is_finite(container_psnr)))
    mean_mse = tf.reduce_mean(tf.boolean_mask((container_mse), tf.math.is_finite(container_mse)))
    std_mse = tf.math.reduce_std(tf.boolean_mask((container_mse), tf.math.is_finite(container_mse)))
    mean_ssim = tf.reduce_mean(tf.boolean_mask((container_ssim), tf.math.is_finite(container_ssim)))
    std_ssim = tf.math.reduce_std(tf.boolean_mask((container_ssim), tf.math.is_finite(container_ssim)))

    print("PSNR on {} set: {} ± {}".format(set_type, (f'{mean_psnr:.4f}'), (f'{std_psnr:.4f}')))
    print("MSE on {} set: {} ± {}".format(set_type, (f'{mean_mse:.4f}'), (f'{std_mse:.4f}')))
    print("SSIM on {} set: {} ± {}".format(set_type, (f'{mean_ssim:.4f}'), (f'{std_ssim:.4f}')))

    if evaluate_tumor_area:
        container_psnr_tumor = container_psnr_tumor.stack()
        container_mse_tumor = container_mse_tumor.stack()
        container_ssim_tumor = container_ssim_tumor.stack()
        threshold = 228
        container_psnr_tumor = tf.boolean_mask((container_psnr_tumor), tf.math.is_finite(container_psnr_tumor))
        container_psnr_tumor = container_psnr_tumor[container_psnr_tumor < threshold]

        mean_psnr_tumor = tf.reduce_mean(container_psnr_tumor)
        std_psnr_tumor = tf.math.reduce_std(container_psnr_tumor)
        mean_mse_tumor = tf.reduce_mean(tf.boolean_mask((container_mse_tumor), tf.math.is_finite(container_mse_tumor)))
        std_mse_tumor = tf.math.reduce_std(tf.boolean_mask((container_mse_tumor), tf.math.is_finite(container_mse_tumor)))
        mean_ssim_tumor = tf.reduce_mean(tf.boolean_mask((container_ssim_tumor), tf.math.is_finite(container_ssim_tumor)))
        std_ssim_tumor = tf.math.reduce_std(tf.boolean_mask((container_ssim_tumor), tf.math.is_finite(container_ssim_tumor)))
        print()
        print("PSNR wrt tumor area on {} set: {} ± {}".format(set_type, (f'{mean_psnr_tumor:.4f}'), (f'{std_psnr_tumor:.4f}')))
        print("MSE wrt tumor area on {} set: {} ± {}".format(set_type, (f'{mean_mse_tumor:.4f}'), (f'{std_mse_tumor:.4f}')))
        print("SSIM wrt tumor area on {} set: {} ± {}".format(set_type, (f'{mean_ssim_tumor:.4f}'), (f'{std_ssim_tumor:.4f}')))

#T1 and T2

In [0]:
evaluate_GAN(dataset=testing, set_type='test', first_sequence=0, second_sequence=1, evaluate_tumor_area=True)

PSNR on test set: 15.3286 ± 4.2134
MSE on test set: 0.0396 ± 0.0275
SSIM on test set: 0.5054 ± 0.2116

PSNR wrt tumor area on test set: 13.6678 ± 3.6085
MSE wrt tumor area on test set: 0.0594 ± 0.0523
SSIM wrt tumor area on test set: 0.9690 ± 0.0449


#T1 and T1c

In [0]:
evaluate_GAN(dataset=testing, set_type='test', first_sequence=0, second_sequence=2, evaluate_tumor_area=True)

PSNR on test set: 23.8431 ± 4.0912
MSE on test set: 0.0058 ± 0.0050
SSIM on test set: 0.8096 ± 0.0984

PSNR wrt tumor area on test set: 20.1544 ± 5.0543
MSE wrt tumor area on test set: 0.0173 ± 0.0216
SSIM wrt tumor area on test set: 0.9833 ± 0.0272


#T1 and T2Flair

In [0]:
evaluate_GAN(dataset=testing, set_type='test', first_sequence=0, second_sequence=3, evaluate_tumor_area=True)

PSNR on test set: 19.9617 ± 4.9249
MSE on test set: 0.0155 ± 0.0131
SSIM on test set: 0.6876 ± 0.1628

PSNR wrt tumor area on test set: 14.6198 ± 3.7807
MSE wrt tumor area on test set: 0.0461 ± 0.0319
SSIM wrt tumor area on test set: 0.9786 ± 0.0324


#T2 and T1c

In [0]:
evaluate_GAN(dataset=testing, set_type='test', first_sequence=1, second_sequence=2, evaluate_tumor_area=True)

PSNR on test set: 15.5055 ± 4.2370
MSE on test set: 0.0387 ± 0.0281
SSIM on test set: 0.5209 ± 0.2061

PSNR wrt tumor area on test set: 13.3221 ± 3.9869
MSE wrt tumor area on test set: 0.0695 ± 0.0718
SSIM wrt tumor area on test set: 0.9679 ± 0.0469


#T2 and T2Flair

In [0]:
evaluate_GAN(dataset=testing, set_type='test', first_sequence=1, second_sequence=3, evaluate_tumor_area=True)

PSNR on test set: 16.8268 ± 3.9727
MSE on test set: 0.0275 ± 0.0199
SSIM on test set: 0.6262 ± 0.1597

PSNR wrt tumor area on test set: 15.1591 ± 4.0428
MSE wrt tumor area on test set: 0.0464 ± 0.0500
SSIM wrt tumor area on test set: 0.9790 ± 0.0309


#T1c and T2Flair

In [0]:
evaluate_GAN(dataset=testing, set_type='test', first_sequence=2, second_sequence=3, evaluate_tumor_area=True)

PSNR on test set: 19.0751 ± 4.2967
MSE on test set: 0.0171 ± 0.0114
SSIM on test set: 0.6562 ± 0.1678

PSNR wrt tumor area on test set: 13.7912 ± 3.9420
MSE wrt tumor area on test set: 0.0607 ± 0.0647
SSIM wrt tumor area on test set: 0.9748 ± 0.0384
